# Repaso $3er $ Parcial 
### Laboratorio de Aprendizaje Estadístico
José Armando Melchor Soto

---


## Temario

1. Arboles de decisión 
    - Regresión 
    - Clasificación
2. Random Forest

3. Gradient boosting
    - xgboost
    - lightgbm
    - catboost
--- 

## Arboles de decisión

- ¿Qué son?

Un árbol de decisión es un modelo predictivo que organiza los datos en distintos niveles jerárquicos, realizando divisiones sucesivas según ciertos valores límite. En cada nivel, separa los datos dependiendo de si cumplen o no con una condición específica, agrupándolos según esas decisiones para llegar a una predicción final.


#### Regresión 

El algoritmo recorre todas las características y todos los posibles umbrales de división. Luego, selecciona el umbral que cause la mayor reducción de varianza.


$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

#### Clasificación

El clasificador considera todas las variables y posibles puntos de corte, pero en este caso selecciona aquel que mejor separa las clases, buscando el umbral que maximice la ganancia de información.

$$
\text{Ganancia de impureza} = \text{Impureza nodo padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

---

## Random Forest

- ¿Qué es?

Es un algoritmo de machine learning basado en árboles de decisión, que crea muchos árboles y combina sus predicciones para mejorar la precisión y evitar el sobreajuste.

#### Regresión 

Se generan varias muestras aleatorias del dataset original usando bootstrap. A partir de esas muestras, se entrenan múltiples árboles. Luego, cada uno hace su predicción y, al final, se toma el promedio de todas las predicciones para obtener el resultado final del modelo.

#### Clasificación

Se entrenan varios árboles con diferentes muestras de datos. Pero en lugar de promediar, se toma la clase que más veces fue predicha por los árboles, es decir, se elige la modal entre todas las predicciones

---

## Gradient Boosting

- ¿Qué es?

Gradient Boosting es un método en machine learning que entrena varios modelos uno tras otro, buscando que cada nuevo modelo corrija los errores del anterior. Hay versiones mejoradas de este enfoque, como XGBoost, LightGBM y CatBoost, que lo hacen más rápido y eficiente.

En estos modelos, la predicción de cada árbol suele ser el promedio de los valores de sus hojas. Y la diferencia entre un regresor y un clasificador es que en el clasificador se usan log-odds para hacer la predicción.

---

#### Xgboost

¿Cómo se calcula XGBoost?

Es un modelo que se construye al sumar un nuevo árbol iterativamente a la predicción. 

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)
$$

Donde el objetivo de cada interación , es que en el nuevo árbol $f_t$ se minimice la contribución marginal a la función de pérdida total.
Esta función objetivo consiste en dos partes: 
- La primera parte es la ya construida:
$$
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)}) + \sum_{k=1}^{t-1} \Omega(f_k)
$$

- La segunda parte es en la que se añade un nuevo árbol $f_t$
$$
\sum_{i=1}^n \left[ l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) - l(y_i, \hat{y}_i^{(t-1)}) \right] + \Omega(f_t)
$$

Quedando como :
$$
\mathcal{L}^{(t)} =
\underbrace{
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)}) + \sum_{k=1}^{t-1} \Omega(f_k)
}_{\text{Parte ya construida (constante en esta iteración)}} +
\underbrace{
\sum_{i=1}^n \left[ l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) - l(y_i, \hat{y}_i^{(t-1)}) \right] + \Omega(f_t)
}_{\text{Lo que añade el nuevo árbol $f_t$}}
$$


Donde $\Omega$ es la regurizacón de la penalización de los árboles.

Al momento de calcularlo nos damos cuenta que es una función complicada, asi que necesitamos aproximarla a un punto usando la expansión de taylor de segundo orden. 
$$
f(x) \approx f(a) + f'(a)(x - a) + \frac{1}{2} f''(a)(x - a)^2
$$

Podemos aplicar el gradiente y el hessiano para poder hacer que la función de perdida se optimice en cada iteración.
$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
$$


Se asigna un valor constante $w_j$ a cada región $R_j$  , esto hace que se pueda regularizar la función de perdida.

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
\right] + \gamma T
$$

Se aplica para cada hoja , ya que anteriormente se minimizó una aproximación cuadrática local de la pérdida.
$$
\text{Output value} = w_j^* = -\frac{G_j}{H_j + \lambda}
$$


Este valor representa el doble de la reducción en la pérdida que se logra con el outputvalue , anteriormente calculado.

$$\text{Similarity Score} = \frac{G_j^2}{H_j + \lambda}$$

Con el valor anterior, podemos decidir si conviene dividir una hoja en un árbol de decisión, se calcula la ganancia del split, que mide la mejora en la calidad del modelo después del split. Si la ganancia es positiva, significa que la pérdida disminuyó.

$$
\text{Gain} = \frac{1}{2} \left( \text{Similarity}_\text{izq} + \text{Similarity}_\text{der} - \text{Similarity}_\text{padre} \right) - \gamma
$$

Y ya para la predicción final, esta se obtiene sumando todos los árboles.
$$
\hat{y}_i = F_0(x_i) + \sum_{t=1}^{M} f_t(x_i)
$$

#### Lightgbm

LightGBM implementa el algoritmo de Gradient Boosting, que entrena múltiples árboles secuenciales donde cada uno corrige los errores del anterior. 

#### Catboost

CatBoost es un algoritmo de machine learning basado en gradient boosting, que destaca por manejar variables categóricas automáticamente y ofrecer alta precisión en tareas de clasificación y regresión.

## Comparativa: XGBoost vs LightGBM vs CatBoost

| Característica              | **XGBoost**                                                | **LightGBM**                                               | **CatBoost**                                                  |
|-----------------------------|------------------------------------------------------------|-------------------------------------------------------------|----------------------------------------------------------------|
| **Velocidad**               | Rápido, pero más lento que LightGBM y CatBoost             | 🔥 Muy rápido gracias a histogramas y leaf-wise growth      | Rápido, aunque un poco más lento que LightGBM                  |
| **Precisión**               | Alta                                                       | Alta, a veces mejor con buen tuning                         | Muy alta, especialmente con categóricas                        |
| **Variables categóricas**   | ❌ No las maneja (requiere encoding manual)                | ❌ No las maneja (requiere encoding manual)                 | ✅ Soporte nativo + regularización secuencial                  |
| **Uso de memoria**          | Moderado                                                   | ✅ Muy eficiente (binning)                                   | Similar a XGBoost                                              |
| **Manejo de missing values**| ✅ Automático                                               | ✅ Automático                                                | ✅ Automático                                                   |
| **Soporte GPU**             | ✅ Sí (bastante estable)                                   | ✅ Sí (muy rápido)                                           | ✅ Sí (algo más limitado)                                      |
| **Instalación**             | Fácil (`pip install xgboost`)                             | Fácil (`pip install lightgbm`)                              | Un poco más pesada (`pip install catboost`)                   |
| **Documentación**           | Excelente                                                  | Buena                                                       | Muy buena                                                     |
| **Interacción con sklearn** | Muy buena                                                  | Muy buena                                                   | Muy buena                                                     |
| **Tolerancia al orden**     | ✅ Neutral                                                  | ✅ Neutral                                                   | ⚠️ Sensible (por codificación secuencial)                      |

---

## ✅ ¿Cuándo usar cada uno?

## ✅ ¿Cuándo usar XGBoost, LightGBM o CatBoost?

| Situación                                                  | Recomendación                                      |
|------------------------------------------------------------|----------------------------------------------------|
| Dataset tabular pequeño o mediano                          | ✅ XGBoost o CatBoost                               |
| Dataset grande, muchas variables numéricas                 | ✅ LightGBM                                         |
| Muchas variables categóricas sin preprocesamiento          | ✅ CatBoost (manejo nativo y robusto)              |
| Quieres algo robusto y estable con buen soporte            | ✅ XGBoost (muy probado en producción y Kaggle)     |
| Entrenamiento rápido con buen desempeño                    | ✅ LightGBM                                         |
| Quieres interpretabilidad con SHAP                         | ✅ Cualquiera, pero CatBoost da mejores resultados con categóricas |
| Necesitas buen rendimiento sin mucho tuning                | ✅ CatBoost (buenos defaults)                       |
| Ya tienes pipeline con OneHot/Target Encoding              | ✅ XGBoost o LightGBM                               |
| Tuning automático (Optuna, GridSearchCV, etc.)             | ✅ LightGBM (rápido y convergente)                  |
| Producción en sistemas legacy o APIs bien documentadas     | ✅ XGBoost (mayor madurez, más integración)         |
| Clasificación multi-label o problemas no estándar          | ✅ XGBoost (soporte más flexible)                   |


## 🧠 Tips

- **LightGBM** puede overfittear fácilmente → cuida `num_leaves` y `min_data_in_leaf`.
- **CatBoost** funciona muy bien con defaults y sin preprocessing.
- **XGBoost** es muy robusto y balanceado, ideal si ya tienes un pipeline con encoding hecho.



La estructura del árbol:

* XGBoost produce árboles más simétricos y balanceados.

* LightGBM produce árboles más profundos y desbalanceados si no se controla.

La precisión y riesgo de overfitting:

* Leaf-wise (LightGBM) puede encontrar mejores divisiones, pero se sobreajusta más fácil.

* Level-wise (XGBoost) es más estable, pero a veces menos preciso.

---

### PREGUNTA EXAMEN 
¿Qué es partial dependence?


Es una técnica que muestra el efecto promedio que tiene una o más variables independientes sobre la predicción de un modelo, al mantener constantes (o promediar) las demás variables.


Ejemplo:

Partial dependence sirve para ver cómo una variable (como las habitaciones de una casa) influye en la predicción del modelo. Si queremos ver cómo cambia el precio de una casa cuando aumentamos las habitaciones, usamos partial dependence para calcular el promedio de los precios predichos por el modelo al ir variando solo las habitaciones y manteniendo el resto igual. Así vemos cómo afecta esa variable específica a la predicción.

#### Clase
Un Partial Dependence Plot (PDP) muestra la relación entre una o más características y la predicción de un modelo, manteniendo las demás variables fijas. Es útil para entender cómo un modelo como un Random Forest toma decisiones.

fig, ax = plt.subplots(figsize=(8, 5))

PartialDependenceDisplay.from_estimator(best_forest, X_train, [7], ax=ax, feature_names=X_train.keys())
